This example shows how to convert pretrained weights from Pytorch to the format that can be loaded by C++ based program, and used by the DLmodelzoo action set as well as dlpy `mzmodel()`. Torchvision 0.8.2 is required to install on the client to run this function. Other versions of Pytorch may work as well, but those versions are not tested.
The function `convert_torchvision_model_weights` takes the parameter `model_name`, where we specify the full name of the model. Currently supported models are:
- resnet(resnet18, resnet34, resnet50, resnet101) 
- vgg(vgg11, vgg13, vgg16, vgg19)
- shufflenet(shufflenet_v2_x0_5, shufflenet_v2_x1_0)
- mobilenet(mobilenet_v2)

In [1]:
import torch
import torchvision

In [2]:
def convert_torchvision_model_weights(model_name, cpp_model_path, python_model_path=None, imsize=(256, 256)):
    '''
    Converts a pretrained model in Python into model weights that can be loaded by C++ programs,
    and used by the MZModel() class. To run this function, the DLPy package is not required.

    Parameters
    ----------
    model_name : string
        Specifies the name of the model.
    python_model_path : string
        Specifies the directory that contains the Python based model weights.
    cpp_model_path : string
        Specifies the derectory to store the C++ based model weights.
    imsize : tuple
        Specifies the dimension of the image passed to the trace function.
    '''
    try:
        temp = getattr(torchvision.models, model_name)
        model = temp(pretrained=True)

        if python_model_path:
            python_weights = torch.load(python_model_path)
            model.load_state_dict(python_weights)
    except:
        raise Exception('Something is wrong while loading the pretrained weights.')

    print('NOTE: Pretrained weights loaded successfully.')

    class wrapped_model(torch.nn.Module):
        def __init__(self)->None:
            super().__init__()
            self.model = model

        def forward(self, x):
            return self.model(x)

    try:
        model_ = wrapped_model()
        img = torch.rand([1, 3, imsize[0], imsize[1]]).float()
        cpp_model = torch.jit.script(model_, img)
        cpp_model.save(cpp_model_path)
    except:
        raise Exception('Something is wrong while saving the pretrained weights.')

    print('NOTE: Pretrained weights saved successfully.')

In [3]:
convert_torchvision_model_weights("resnet18", "/path/to/resnet18_cpp.pt")

NOTE: Pretrained weights loaded successfully.
NOTE: Pretrained weights saved successfully.


In [4]:
import swat as sw
from dlpy.mzmodel import *
from dlpy.utils import *

In [5]:
# Connect to an existing CAS server
s = sw.CAS('your-server.unx.company.com',2999)

In [6]:
s.table.addcaslib(activeonadd=False,
                  datasource={'srctype':'path'},
                  name='dnfs',
                  path='/example/path',
                  subdirectories=True)

NOTE: Cloud Analytic Services added the caslib 'dnfs'.


,Name,Type,Description,Path,Definition,Subdirs,Local,Active,Personal,Hidden,Transient
0,dnfs,PATH,,/example/path/,,1.0,1.0,0.0,0.0,0.0,0.0


In [7]:
s.table.loadTable(caslib='dnfs', path='data/cifar10_small.sashdat', 
    casout = dict(name = 'data', blocksize = '1', replace = True))

NOTE: Cloud Analytic Services made the file data/cifar10_small.sashdat available as table DATA in caslib CASUSER(xingdu).


[caslib]

 'CASUSER(xingdu)'

[tableName]

 'DATA'

[casTable]

 CASTable('DATA', caslib='CASUSER(xingdu)')

+ Elapsed: 0.00769s, user: 0.00598s, mem: 0.854mb

In [8]:
model = MZModel(conn=s, model_type = "torchNative", model_name="resnet", model_subtype="resnet18", 
                num_classes=10, model_path = "/path/to/resnet18_cpp.pt")

In [9]:
model.add_image_transformation(image_size='256 256')

In [10]:
optimizer=Optimizer(seed=54321, 
                    algorithm=SGDSolver(lr=0.001, momentum=0.9),
                    batch_size=128,
                    max_epochs=10                   
                    )

In [11]:
model.train(table="data", inputs="_image_", targets="labels", gpu=[0], optimizer=optimizer, index_variable='labels')

NOTE: Using this GPU List on your-server.unx.company.com: [0 ].
NOTE: Worker rank 0: your-server.unx.company.com, using up to 1 GPU devices.
NOTE: Using your-server.unx.company.com: GPU 0 processes up to 128 records at a time.
NOTE: Starting dlxexe process on your-server.unx.company.com, rank/threadID [0/0], pid= 2124 device= GPU:0
NOTE: Learning Rate:   0.001 Momentum:     0.9 Weight Decay:       0 Dampening:       0 BatchSize: 128 Rank: 0 Device: 0 Loss: 135.924
NOTE: The action completed successfully.


,Epoch,Loss,MCE
0,0.0,1575.153782,0.927536
1,1.0,1180.905790,0.865566
2,2.0,898.110307,0.782278
3,3.0,709.094187,0.621776
4,4.0,548.199011,0.489796
5,5.0,403.487586,0.324042
6,6.0,291.951439,0.135659
7,7.0,220.007848,0.064386
8,8.0,170.034453,0.040733
9,9.0,135.924037,0.024641
